# Setup OpenAI Key

In [ ]:
!pip install -q openai tiktoken

In [ ]:
import openai
import os
import tiktoken
import getpass

api_key = getpass.getpass("Enter the OpenAI API Key: ")
assert api_key.startswith("sk-"), 'OpenAI API Keys begin with "sk-".'
openai.api_key = api_key

Enter the OpenAI API Key: ··········


# Write the prompt

In [ ]:
enc = tiktoken.encoding_for_model("gpt-3.5-turbo")

emotions = [
    "claim",
    "premise",
]

def create_logit_biases(emotions, bias_weight):
    """Creates a dict of tokens for ChatGPT to use mostly exclusively."""
    bias_dict = {}
    for emotion in emotions:
        token_ids = enc.encode(emotion)
        for token_id in token_ids:
            bias_dict[token_id] = bias_weight
    return bias_dict

def chatgpt_sentiment(prompt, emotions=None):
    if not emotions:
        emotions = globals().get("emotions")
    emotion_list_str = [f"- {x}\n" for x in emotions]
    eos_str = "."
    #emotionally
    system = f"You are a logician. Classify the sentiment of the user's text with ONLY ONE OF THE FOLLOWING CLASSIFICATION:\n{''.join(emotion_list_str)}\n\nAfter classifying a text, respond with \"{eos_str}\"."
    r = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": system},
            {"role": "user", "content": prompt},
        ],
        stop=eos_str,
        max_tokens=20,  # safeguard from going infinite
        temperature=0.0,  # deterministic, will use largest logit
        logit_bias=create_logit_biases(
            emotions + [eos_str], 15  # may want to tweak bias weight
        ),
    )

    result = r["choices"][0]["message"]["content"]
    if result == "":  # if ChatGPT decides to not return an emotion
        result = "N/A"
    return result

# Generate The Sentiment
Test Cases: 
Class 0: Premise.
Class 1: Claim.
Input the text you want to classify in the cell below, then run the cell!

In [ ]:
text = "And in that context, of course, they're lifting and shifting some of the older workloads, but they're modernizing the entire business process flow." #@param {type:"string"}
chatgpt_sentiment(text)

'premise'

In [ ]:
chatgpt_sentiment("It's a number that is incredibly competitive in our industry, and we want to continue to keep it that way.")

'claim'

In [ ]:
chatgpt_sentiment("Even while in International, we're continuing to invest in a lot of areas, we continue to frontload Prime benefits for the newer geographies, we continue to launch new countries as we launch Prime in Australia recently.")

'claim'

In [ ]:
chatgpt_sentiment("See, first of all, I'd say the opportunity for our shareholders when they think about Microsoft has never been better.")

'claim'

In [ ]:
chatgpt_sentiment("So this is an architectural design point that we have built from the ground up from day one and it's good to see people validating it now and elsewhere and we will take that as a validation of something that we thought of a long time ago.")

'claim'

# Save json files

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import json

with open('/content/drive/MyDrive/ECC_Argument_Classification_Test.json', 'r', encoding='utf-8') as f:
    original_data = json.load(f)

sentences = [item['sentence'] for item in original_data]

results = []
for sentence in sentences[:60]:
    result = chatgpt_sentiment(sentence)
    if result == "claim":
        sentiment = 1
    elif result == "premise":
        sentiment = 0
    else:
        sentiment = -1
    results.append(sentiment)

output_data = []
for item, sentiment in zip(original_data, results):
    item['Prediction'] = sentiment
    output_data.append(item)

with open('/content/drive/MyDrive/ECC_Argument_Classification_Test60.json', 'w', encoding='utf-8') as f:
    json.dump(output_data, f, ensure_ascii=False, indent=4)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import json

# Load JSON file
with open('/content/drive/MyDrive/ECC_Argument_Classification_Test.json', 'r', encoding='utf-8') as f:
    original_data = json.load(f)

# Extract sentences
sentences = [item['sentence'] for item in original_data[900:970]]

results = []
for sentence in sentences:
    result = chatgpt_sentiment(sentence)
    if result == "claim":
        sentiment = 1
    elif result == "premise":
        sentiment = 0
    else:
        sentiment = -1  # If sentiment is unrecognizable, then -1
    results.append(sentiment)

# Create a list of dictionaries containing sentences and sentiment results, and retain the original JSON structure
output_data = []
for item, sentiment in zip(original_data[900:970], results):
    item['Prediction'] = sentiment
    output_data.append(item)

# Append the results to the existing JSON file
with open('/content/drive/MyDrive/ECC_Argument_Classification_Test970.json', 'a', encoding='utf-8') as f:
    for item in output_data:
        json.dump(item, f, ensure_ascii=False)
        f.write(',\n')